# Day 4: データ可視化
Day 4では，関係データベースなどのデータセットから抽出・集約したデータを使って，効果的な説明・考察を行うためのデータ可視化方法に関する演習を行う．
演習の目的は，データ可視化のためのツールの修得ではない．目的に応じた適切なデータ可視化手法を理解することにある．
そのため，Day 4では（データ可視化のためのツールの使い方を体験する）ドリルは設けておらず，レポート課題のみを用意した．
レポート課題で扱うデータは，仮想の小売店（杏森堂）の関係データベースに保存された購買データである．

データ可視化のためのツールは特に指定しない．データ可視化を行うツールとして，
* 手軽ものとしてはExcel，
* 可視化の手続きをコード化できるものとしてRのggplot2，Pythonのmatplotlib，plotlyなど

があるが，どのツールを利用しても問題はない．


なお，Day 2からDay 5の演習で用いるデータセットは，[データサイエンス100本ノック（構造化データ加工編）](https://github.com/The-Japan-DataScientist-Society/100knocks-preprocess)で提供されているデータセットをSQLite形式に変換したものである．「課題」は完全に山本が作成したものである．

---

## SQL利用環境セットアップ
SQLiteはWindows/Mac/LinuxどのOSでもインストールして利用することができる．本演習は環境構築の手間をできるだけ減らすために，Google Colaboratory上でSQLiteを用いる．

Googleにログインした状態で[本演習サイト](https://data-analytics2022.hontolab.org)の各回の「ドリル（模範解答なし）」のリンクをクリックすると，あなた用のGoogle Colaboratoyがブラウザ上で立ち上がり，各回のドリル・課題を行うためのページが表示される．
ページが表示されたら，以下のコードが書かれた箇所（セル）を実行しよう．すると，Google Colaboratory上でSQLiteを実行するための環境が用意される．

なお，Google Colaboratory上でコードを実行するには，以下の2つの方法がある：
* 「再生ボタン」をクリックする
* コードが書かれたセルをクリックし，Shift + Enterキーを押す

In [ ]:
# Google Colaboratory上のSQLiteをアップグレードする
import sys
if 'google.colab' in sys.modules:
    !gdown --id 1BSHIKQ7rFw5BpTq5nw1UZfjPK_7Mpnbi -O /usr/lib/python3.7/lib-dynload/

上記コードを実行したら，アップグレード版SQLiteを使うために，Google Colaboratoryを再起動させよう．
再起動の方法は，画面上部の`Runtime`から`Restart Runtime`をクリックする．

再起動が完了したら，コードを実行しよう．以下のコードはGoogle Colab（Jupyter）上でSQLを対話的に利用するためのおまじないである．

In [ ]:
# ipython-sqlがインストールされていない場合
!pip install ipython-sql

英文が数行表示された後，SQLiteがインストールされる． 

続いて，演習で用いる購買データが収められたSQLiteデータベースをダウンロードしよう． 以下のコードを実行すると，Google Colaboratory上にデータベースがダウンロードされる．

In [ ]:
# SQLiteデータベース（data-analytics-lecture.db）のファイルをダウンロード
import os
if not os.path.exists('data/data-analytics-lecture.db'):
    !wget -P data https://github.com/hontolab-courses/data-analytics-2022/raw/main/data/data-analytics-lecture.db

データベースがダウンロードできたら，データをロードしておこう．
以下のコードを実行すると，Google Colaboratory上でダウンロードしたSQLiteデータベースを操作できるようになる．

In [ ]:
%load_ext sql
%sql sqlite:///data/data-analytics-lecture.db

なお，Google Colaboratoryはセッションを閉じると，環境が初期化され，以前に自前でインストールしたライブラリが削除される．
Google Colaboratoryを使って演習に取り組む際には，上記コードを毎回忘れず実行しよう．

---

## Google Colaboratory上でのSQLの実行方法

Google Colaboratoryでは，以下のようにセルの冒頭に`%%sql`と書き，2行目以降にSQL文を書くとSQLを実行することができる．
試しに以下のSQLを実行してみよう．

In [ ]:
%%sql

SELECT
    name AS table_name
FROM
    sqlite_master
WHERE
    type = 'table';

上記SQLは，データベースに格納されたテーブルの一覧を表示するものである．
ダウンロードしたSQLiteデータベース，つまり今回扱う杏森堂のデータベースには，**6つ**のテーブルが格納されていることが分かる．なお，対象データベースの構造が記された実体関連図（ER図）は[コチラ](https://github.com/hontolab-courses/data-analytics-2022/raw/main/doc/ER-diagram.pdf)から確認することできる．

課題とドリルに取り組む前に，あと1つ，別のSQLを実行してみよう．
以下のSQLを実行すると，`receipt`テーブルの最初の10件を表示することができる
（SQL文に`LIMIT`を付けないと，テーブル内の全データが表示されてしまうので注意）．

In [ ]:
%%sql

SELECT * FROM receipt LIMIT 10; 

## ★ SQLの実行結果をCSVに保存したい場合
以下のように，セルの冒頭に`%%sql 一時保存先の変数名 <<`と書き，2行目以降にSQL文を書いて実行しよう．

In [ ]:
%%sql records <<

SELECT * FROM receipt LIMIT 10; 

その後以下のコマンドを実行することで，SQLの実行結果を格納したCSVファイルを作成することができる．作成したCSVファイルは，画面左のサイドバーにあるファイル一覧からダウンロードできる．

In [ ]:
# records.csvは任意のファイル名にできる
records.csv(filename="data/records.csv")

---

## 課題4

### 課題4-1
顧客ごとに購買頻度（購買回数）を求め，購買頻度の分布を説明するためのグラフを作成せよ．また，グラフから読み取れることを文章で記述せよ．ただし，顧客ID（`customer_id`）が"Z"から始まるのものは非会員を表すため，除外して分析すること．

なおレポートには，顧客ごとに購買頻度（購買回数）を求めるためのSQL文も付しておくこと（SQL文の実行結果は掲載する必要なし）．

In [ ]:
%%sql


### 課題4-2
都道府県別に年月別の売上総額を求め，その売上総額の変化を比較するためのグラフを作成せよ．また，グラフから読み取れることを文章で記述せよ．

なおレポートには，都道府県別の年月別売上総額を求めるためのSQL文も付しておくこと（SQL文の実行結果は掲載する必要なし）．

In [ ]:
%%sql


### 課題4-3
顧客一人あたりの売上総額の分布を，千葉県の店舗ごとに比較するためのグラフを作成せよ．また，グラフから読み取れることを文章で記述せよ．ただし，顧客ID（`customer_id`）が"Z"から始まるのものは非会員を表すため，除外して分析すること．

なおレポートには，店舗ごとにその店舗を利用した各顧客の売上総額を集計するSQL文も付しておくこと（SQL文の実行結果は掲載する必要なし）．

In [ ]:
%%sql


### 課題4-4
千葉県にある店舗間で顧客が購入する菓子の傾向が異なるかどうかを分析したい．千葉県にある店舗と菓子の中カテゴリ（cateogory_medium_cd）ごとに菓子の購入量（個数）を集計し，購入された菓子の中カテゴリの内訳を店舗間で比較するためのグラフを作成せよ．また，グラフから読み取れることを文章で記述せよ．

なおレポートには，購入量を集計するためのSQL文も付しておくこと（SQL文の実行結果は掲載する必要なし）．

In [ ]:
%%sql
